________________________________________________________________________________________________________
SCRAPE DATA USING SELENIUM AND BEAUTIFUL SOUP 4

STORE DATA INTO FIREBASE DATABASE
________________________________________________________________________________________________________

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

import firebase_admin
from firebase_admin import credentials, firestore
from firebase_admin import db


# Initialize Firebase Admin SDK with service account credentials
cred = credentials.Certificate("C:/Users/izzat/OneDrive/Desktop/MindHive Assignment/subway-scraper-firebase-adminsdk-y1xwg-647b4207ad.json")
# firebase_admin.initialize_app(cred)

# Set up Selenium WebDriver
driver = webdriver.Chrome()
driver.get('https://subway.com.my/find-a-subway')

# Scroll through the sidebar
# Replace '1000' with the desired scroll height
driver.execute_script("document.querySelector('.sidebar').scrollTop = 1000")

# Enter search query into the search bar
search_input = driver.find_element(By.CSS_SELECTOR, '#fp_searchAddress')
search_input.clear()
search_input.send_keys('Kuala Lumpur' + Keys.ENTER)

# Wait for the results to load
time.sleep(5)

places = driver.find_element(By.CLASS_NAME, 'fp_ll_holder')
# print (places.get_attribute("outerHTML"))
elementSoup = BeautifulSoup(places.get_attribute("outerHTML"),'html.parser')
# print(elementSoup)
direction_buttons = elementSoup.find_all(class_="directionButton")

# Access Firestore
db = firestore.client()

# Get a reference to the Firestore collection
subway_locations_ref = db.collection('subway_locations')

for my_tag in elementSoup.find_all(class_="fp_listitem") :
    if "Kuala Lumpur" in my_tag.text:
        # Split the text based on ","
        name, rest_of_text = my_tag.text.split(",", 1)
        
        # Split the address and hours from the word "Monday"
        index_monday = rest_of_text.find("Monday")
        address = rest_of_text[:index_monday].strip()
        
        # Extract the operating hours
        operating_hours = rest_of_text[index_monday:].split("Find out more")[0].strip()
        
        # Find the anchor tag inside the "directionButton" element
        anchor_tag = my_tag.find_all("a")
        # print(anchor_tag)
        
        print("Name:", name.strip())
        print("Address:", address.strip())
        print("Operating Hours:", operating_hours.strip())

        # Check if the anchor tag exists and extract the href attribute (Waze link)
        if len(anchor_tag) > 2:
            waze_link = anchor_tag[2].get("href")
            print("Waze Link:", waze_link)
        else:
            print("Waze Link not found")
        print()

        # # Create a document in the Firestore collection
        # subway_locations_ref.add({
        #     'name': name.strip(),
        #     'address': address.strip(),
        #     'operating_hours': operating_hours.strip(),
        #     'waze_link': waze_link
        # })


# Close the WebDriver
driver.quit()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/izzat/OneDrive/Desktop/MindHive Assignment/subway-scraper-firebase-adminsdk-y1xwg-647b4207ad.json'